# **Set Analyzer**

Le but de ce script est d'analyser statistiquement les différents set de carte du jeu *Magic the Gathering* afin de :


1.   Déterminer les cartes les plus pertinentes dans chaque set
2.   Comparer les métadonnées liées aux cartes entre chaque set

L'objet de l'analyse est de pouvoir objectiver la valeur intrinsèque d'une carte *intra* et *inter* set.

L'application visée est l'utilisation de ses données afin de performer dans les formats limités (*sealed, draft*)




***draft***

quelques métadonnées utiles

**intraset**
- nombre de cartes + ratios
- card type : couleur, créature / non-créature, rareté
- cout : valeur de mana
- board : power, toughness, evasion (en fonction de keywords: flying, trample, menace, ...)
- interaction : (en fonction de keywords: counter, return, deal damage, ...)
- mana fixing : type de manafix (en fonction de keywords: dork, rock, treasures, ...)

**interset**
- replacer chacune des données précédentes par rapport aux précédents sets
- évaluer : vitesse du set, créatures fragiles, propension de bombes, propension d'interaction, potentiel de fixing, etc





# **Initialisation**

In [ ]:
# Import librairies to use in the code

import os
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

Charger le set de cartes

dataset provient de : https://mtgjson.com/

In [ ]:
# set path of the folder containing dataset
path=os.getcwd()+'/drive/MyDrive/Colab Notebooks/_MTG/_data/'

# set pathfile
file = 'AllPrintings.json'
pathfile = path + file

In [ ]:
# load all dataset
data = pd.read_json(pathfile)
AllSet = data.iloc[2:]['data'] # 2 first rows of JSON files are metadata

Select type of analysis
- Intraset : for one set only
- Interset : to compare with other sets

In [ ]:

# specific_set_trig = input("Analyze specific set ? \n Yes \n No \n")

# match specific_set_trig:
#   case "Yes":
#     set_name = input("Type set code \n")
#   case "No":
#     pass

#only as an example
set_name = 'OTJ'


# **Intraset**

In [ ]:
#load cards of a specific set in a DataFrame
#example : Outlaws of Thunder Junction (OTJ)

df = pd.DataFrame.from_dict(AllSet.loc[set_name]['cards'])
features_analyzed = [
    'name',
    'keywords',
    'manaValue',
    'colorIdentity',
    'power',
    'toughness',
    'rarity',
    'types',
    'text'
]
cards = df.loc[:AllSet.loc[set_name]['baseSetSize']-1, features_analyzed]

cards


## Clean data

Check if:
- `manaValue` is numeric (some cards may have no manaValue ?)
- `power` and `toughness` are numeric (these stats may be variables and notes as a `'*'` symbol)


In [ ]:
# clean manaValue, power and toughness
cards['manaValue'] = pd.to_numeric(cards['manaValue'], errors='coerce')
cards['power'] = pd.to_numeric(cards['power'], errors='coerce')
cards['toughness'] = pd.to_numeric(cards['toughness'], errors='coerce')

In [ ]:
# keep only common and uncommon cards
cards_unco = cards[cards['rarity'].isin(['common', 'uncommon'])]

******** SANDBOX *******

In [ ]:
### clean

## transform list of string in string in the types column
# a = cards.types[0]
# print(a)

# b = a[0]
# print(b)
# type(b)

# c = cards.types[0][0]
# print(c)

## search for all lands
# cards[cards['types'].apply(lambda x: 'Land' in x)]

## search for all white and blue cards
# cards[cards['colorIdentity'].apply(lambda x: ['U', 'W']==x)]

## search for all cards with two or more keywords in lists
# def two_plus_keyword(lst):
#   if isinstance(lst,list) and len(lst)>1:
#       return True
#   else:
#     return False

# cards[cards['colorIdentity'].apply(two_plus_keyword)]
# cards[cards['types'].apply(two_plus_keyword)]

**** END OF SANDBOX *****

## 1) Vitesse du format

La vitesse du format peut être déterminée par :
- le nombre de créature (ou le ratio) => mettre une note (S à D, en fonction du % ration, découper en 6 section: exemple 100-90 == S ; 90 - 70% == A ; 70 - 50% == B ; 50 - 30% == C ; 30 - 10% == D ; 10 - 0% == E)
- le nombre d'interactions (ou le ratio) => idem
- la valeur moyenne de `mana value` des créatures => afficher un histogramme
- le `power` des créatures vs. `mana value` => afficher graph power vs. manavalue
- le `power`a comparé a la `toughness` => est ce que les creatures sont à même de bloquer l'état du board ou est ce qu'il y a plus de trades ? (peut aussi être comparé par `mana value`)





In [ ]:
# 1) Speed

cards_unco_crea = cards_unco[cards_unco['types'].apply(lambda x: 'Creature' in x)] #all common / uncommon creatures

# 1a) le nombre + ratio de creatures
n_unco_tot = len(cards_unco)
n_unco_crea = len(cards_unco_crea)
ratio_unco_crea = (n_unco_crea / n_unco_tot) * 100

# 1b) le nombre + ratio d'interactions
# TBD

# 1c) Manavalue des créatures
[manaValue_mean, manaValue_std] = [cards_unco_crea['manaValue'].mean(), cards_unco_crea['manaValue'].std()]

# 1d) Power vs. manavalue des créatures
ratio_power_to_mana = (cards_unco_crea['power'] / cards_unco_crea['manaValue'])
[ratio_power_to_mana_mean, ratio_power_to_mana_std] = [ratio_power_to_mana.mean(), ratio_power_to_mana.std()]

In [ ]:
# *** FIGURES ***

# 1a)
print('ratio of creatures = ' + str(ratio_unco_crea) + ' %')

# 1c)
plt.subplot(2,1,1)
plt.hist(cards_unco_crea['manaValue'], bins=np.arange(-0.5,max(cards_unco_crea['manaValue'])+0.5,1))

plt.xlabel('manaValue')
plt.ylabel('number or cards')

plt.axvline(manaValue_mean, ls='--', color='r')
plt.text(6,20,'mean value = '+str(manaValue_mean), color='r')

# 1d)
plt.subplot(2,1,2)
# plt.plot(cards_unco_crea['power'], cards_unco_crea['manaValue'], 'o', color='b')
counts = cards_unco_crea.groupby(['manaValue', 'power']).size().reset_index(name='count')
plt.scatter(counts['manaValue'], counts['power'], s=counts['count']*100, alpha=0.5, edgecolors='w', color='b')

plt.xlabel('manaValue')
plt.ylabel('power')
for i in range(len(counts)):
    plt.text(counts['manaValue'][i], counts['power'][i], str(counts['count'][i]), fontsize=12, ha='center', va='center')

plt.plot(range(0,7), range(0,7), color='r', ls='--')

plt.show()

## 2) BOARD STATE

Quel est le board "type" ?
- taille des créatures : power + toughness
- importance du combat : power vs. toughness, y-a-til un threshold de toughness pour lequel, il est difficile de combattre ? (X% des cartes de "passent pas" => cumulated distribution)
- importance de l'évasion : nombre/ratio de créatures évasives par mot-clé

In [ ]:
# 2) BOARD STATE

# 2a) creatures' power & toughness
[power_mean, toughness_mean] = cards_unco_crea[['power', 'toughness']].mean()
[power_std, toughness_std] = cards_unco_crea[['power', 'toughness']].std()

# 2b) importance du combat

# 2c) evasion
all_unique_keywords = list(cards_unco_crea['keywords'].explode().unique())
all_unique_keywords.remove(np.nan)
targeted_keywords = ['Flying', 'Trample', 'Menace']

df = cards_unco_crea
exploded_df = df.explode('keywords')
filtered_df = exploded_df[exploded_df['keywords'].isin(targeted_keywords)]
filtered_df['keywords'].value_counts()

In [ ]:
# *** FIGURES ***

# 2a) + 2b)
data_power = cards_unco_crea['power']
data_toughness = cards_unco_crea['toughness']
max_of_series = max(max(cards_unco_crea['power']), max(cards_unco_crea['toughness']))
bins = np.arange(-0.5,max_of_series+0.5,1)

fig1, ax1 = plt.subplots(1, 1, figsize=(6, 6))
ax2 = ax1.twinx()
plt.hist(data_power, bins=bins, color='skyblue')
[counts_power, edges_power] = np.histogram(data_power, bins=bins)
cum_power = np.cumsum(counts_power) / np.sum(counts_power) * 100
ax2.plot(np.arange(0,max_of_series,1), cum_power, ls='--', color='b')

fig2, ax1 = plt.subplots(1, 1, figsize=(6, 6))
ax2 = ax1.twinx()
plt.hist(data_toughness, bins=bins, color='salmon')
[counts_toughness, edges_toughness] = np.histogram(data_toughness, bins=bins)
cum_toughness = np.cumsum(counts_toughness) / np.sum(counts_toughness) * 100
ax2.plot(np.arange(0,max_of_series,1), cum_toughness, ls='--', color='r')




# fig, ax = plt.subplots(figsize=(10, 8))
# counts_left, edges_left = np.histogram(data_left, bins=bins)
# ax.barh(edges_left[:-1], -counts_left, height=np.diff(edges_left), color='skyblue', alpha=0.7, label='power')

# counts_right, edges_right = np.histogram(data_right, bins=bins)
# ax.barh(edges_right[:-1], counts_right, height=np.diff(edges_right), color='salmon', alpha=0.7, label='toughness')

# max_freq = max(counts_left.max(), counts_right.max())
# ax.set_xlim(-max_freq - 1, max_freq + 1)
# ax.legend(loc="upper right")

# # 2b)

# # Calculate cumulative percentages for both sides
# cum_left = np.cumsum(counts_left) / np.sum(counts_left) * 100
# cum_right = np.cumsum(counts_right) / np.sum(counts_right) * 100

# # Plot cumulative curves for each side
# ax.plot(-cum_left, edges_left[:-1] + np.diff(edges_left) / 2, color='blue', linestyle='--', linewidth=2, label='Cumulative % (Variable 1)')
# ax.plot(cum_right, edges_right[:-1] + np.diff(edges_right) / 2, color='red', linestyle='--', linewidth=2, label='Cumulative % (Variable 2)')



## 3) Interactions

a quel point le set est interactif ?
000 - définir ce qu'est une interaction
- ratio de permanents
- pourcentage d'interaction
- la "vitesse" de l'interaction = distribution de mana value des sorts interactifs
- type d'interaction : single-target removal + combat trick
- color pie

In [ ]:
# ratio of permanents

def checklist(items_wanted, items_tbc):
  return any(item in items_wanted for item in items_tbc)

permanent_index = [item for item in type_index if (item !='Instant' and item!='Sorcery')]
cards['types'][cards['types'].apply(lambda x: checklist(x,permanent_index)==False)] #non-permanent
cards['types'][cards['types'].apply(lambda x: checklist(x,permanent_index)==True)]  #permanents

print('permanent ratio = ' + str(len(cards['types'][cards['types'].apply(lambda x: checklist(x,permanent_index)==True)])/len(cards)*100) + ' %')

In [ ]:
# get interactive cards

interaction_list = [
    'destroy',
    'exile',
    'counter',
    'target'
]




def interactive_card(str):
  if any(word in str for word in interaction_list):
    return True
  else:
    return False

cards[cards['text'].apply(interactive_card)]

## 4) Fixers

a quel point une carte d'une autre couleur est facile à jouer ? => tout mettre sur une color pie
- carte qui produisent du mana coloré : dorks, rock, treasures
- type de mana produit
- cartes à plusieurs pips (en % des cartes)


# **Interset**

In [ ]:
# toutes les stats interset

# mettre en input le nombre et la temporalité des sets à comparer (tout le modern, 4 derniers sets, etc)
# écrire une ligne d'input

# appeler les fonctions précédentes
# ranger dans des listes / df pour faire les statistiques ensuite